In [1]:
import pandas as pd
import numpy as np
import re
import csv
import matplotlib.pyplot as plt
import os
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

### Cleaning Functions

In [2]:
def show_basic_info(dataframe: pd.DataFrame) -> None:
  """
  Shows basic information for the given dataframe.

    Parameters
    ----------
    dataframe : pd.DataFrame
        The dataframe to be processed

  """
  print(f'Basic shape (rows, columns): {dataframe.shape}')

  print(f'\nColumns present in the data frame: {dataframe.columns}')

  print(f'\nIndex of the data frame: {dataframe.index}')

  print('\nGeneral dataframe information: ')
  dataframe.info()

  print('\nFirst 10 lines of the data frame:')
  display(dataframe.head(10))

  print(f'\nBasic statistics:')
  display(dataframe.describe())
  
  
def clean_column_names(df):
    # Define a translation map for accented to unaccented vowels
    accents = str.maketrans('áéíóú', 'aeiou')

    # Replace accents, clean, and capitalize column names
    df.columns = [
        col.translate(accents).replace(" ", "_").replace(",", "").capitalize()
        for col in df.columns
    ]

    # Replace accents in all string values of the DataFrame
    df = df.applymap(lambda x: x.translate(accents) if isinstance(x, str) else x)

    return df
  
def count_nan_per_column(df):
    # Count NaN values per column
    nan_count = df.isna().sum()
    # Calculate the percentage of NaN values
    nan_percentage = (nan_count / len(df)) * 100
    # Combine the counts and percentages into a DataFrame
    nan_count_df = pd.DataFrame({
        'NaN Count': nan_count,
        '%': nan_percentage
    })
    return nan_count_df

def get_unique_values(df, column_name):
    """
    Returns the unique values from a specified column in the DataFrame.

    Parameters:
    - df (DataFrame): The input DataFrame.
    - column_name (str): The name of the column for which to extract unique values.

    Returns:
    - List: A list of unique values from the specified column.
    """
    # Ensure the specified column exists
    if column_name not in df.columns:
        raise ValueError(f"The DataFrame must contain a '{column_name}' column.")
    
    # Get unique values from the specified column
    unique_values = df[column_name].unique().tolist()
    
    return unique_values

def drop_nan_rows(df, column_name=None):
    """
    Drops rows with NaN or equivalent missing values from the DataFrame.
    
    Parameters:
    - df (DataFrame): The input DataFrame.
    - column_name (str, optional): The column to check for NaN values. 
                                   If None, drops rows with NaN values in any column.
    
    Returns:
    - DataFrame: A new DataFrame with rows containing NaN or equivalent missing values dropped.
    """
    # Replace non-standard missing values like 'NaN', 'null', '' with np.nan
    df = df.replace(['NaN', 'null', 'NULL', ''], np.nan)

    if column_name is None:
        # Drop rows with NaN in any column
        return df.dropna(how='any')
    else:
        # Ensure the specified column exists
        if column_name not in df.columns:
            raise ValueError(f"The DataFrame must contain a '{column_name}' column.")
        # Drop rows with NaN in the specified column
        return df.dropna(subset=[column_name])

### Downloading Dataset
**Only run once a day**

In [3]:
import os
from kaggle.api.kaggle_api_extended import KaggleApi

# Authenticate with Kaggle API
api = KaggleApi()
api.authenticate()

# Download the dataset
dataset_name = "asaniczka/top-spotify-songs-in-73-countries-daily-updated"
destination = "/Users/ikermontane/Documents/Spotify"  # Specify the destination folder

# Ensure the destination folder exists
if not os.path.exists(destination):
    os.makedirs(destination)

# Download and unzip the dataset
api.dataset_download_files(dataset_name, path=destination, unzip=True)

print(f"Dataset downloaded to: {destination}")

Dataset URL: https://www.kaggle.com/datasets/asaniczka/top-spotify-songs-in-73-countries-daily-updated
Dataset downloaded to: /Users/ikermontane/Documents/Spotify


In [4]:
path = "/Users/ikermontane/Documents/Spotify/universal_top_spotify_songs.csv"

df = pd.read_csv(path)

In [5]:
show_basic_info(df)

Basic shape (rows, columns): (1609451, 25)

Columns present in the data frame: Index(['spotify_id', 'name', 'artists', 'daily_rank', 'daily_movement',
       'weekly_movement', 'country', 'snapshot_date', 'popularity',
       'is_explicit', 'duration_ms', 'album_name', 'album_release_date',
       'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'time_signature'],
      dtype='object')

Index of the data frame: RangeIndex(start=0, stop=1609451, step=1)

General dataframe information: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1609451 entries, 0 to 1609450
Data columns (total 25 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   spotify_id          1609451 non-null  object 
 1   name                1609421 non-null  object 
 2   artists             1609422 non-null  object 
 3   daily_rank          1609451 non-null

,spotify_id,name,artists,daily_rank,daily_movement,weekly_movement,country,snapshot_date,popularity,is_explicit,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,3sK8wGT43QFpWrvNQsrQya,DtMF,Bad Bunny,1,0,8,NaN,2025-01-15,91,True,...,7,-27.405,0,0.0717,0.1770,0.2180,0.0807,0.03200,136.020,4
1,2plbrEY59IikOBgBGLjaoe,Die With A Smile,"Lady Gaga, Bruno Mars",2,0,-1,NaN,2025-01-15,100,False,...,6,-7.777,0,0.0304,0.3080,0.0000,0.1220,0.53500,157.969,3
2,4wJ5Qq0jBN4ajy7ouZIV1c,APT.,"ROSÉ, Bruno Mars",3,0,-1,NaN,2025-01-15,90,False,...,0,-4.477,0,0.2600,0.0283,0.0000,0.3550,0.93900,149.027,4
3,2lTm559tuIvatlT1u0JYG2,BAILE INoLVIDABLE,Bad Bunny,4,0,2,NaN,2025-01-15,90,True,...,10,-46.113,1,0.0615,0.1920,0.7900,0.1120,0.21900,119.387,3
4,5TFD2bmFKGhoCRbX61nXY5,NUEVAYoL,Bad Bunny,5,0,0,NaN,2025-01-15,90,False,...,6,-20.024,1,0.1390,0.2650,0.9950,0.2040,0.12000,137.922,4
5,7ne4VBA60CxGM75vw0EYad,That’s So True,Gracie Abrams,6,0,-3,NaN,2025-01-15,96,True,...,1,-4.169,1,0.0368,0.2140,0.0000,0.1590,0.37200,108.548,4
6,6dOtVTDdiauQNBQEDOtlAB,BIRDS OF A FEATHER,Billie Eilish,7,0,-3,NaN,2025-01-15,97,False,...,2,-10.171,1,0.0358,0.2000,0.0608,0.1170,0.43800,104.978,4
7,59D4DOkspUbWyMmbAPQkxZ,VOY A LLeVARTE PA PR,Bad Bunny,8,0,-1,NaN,2025-01-15,89,True,...,7,-35.032,1,0.0588,0.9210,0.4530,0.4050,0.00001,106.295,3
8,7d6yK8v8J484SWH5prIQiE,VeLDÁ,"Bad Bunny, Omar Courtz, Dei V",9,0,-1,NaN,2025-01-15,88,True,...,1,-18.638,1,0.1490,0.2900,0.0023,0.4370,0.03370,101.852,3
9,5WEF0icHWmAZBBMglBd599,WELTiTA,"Bad Bunny, Chuwi",10,0,5,NaN,2025-01-15,87,False,...,4,-32.354,0,0.6000,0.8660,0.3290,0.3460,0.35700,96.140,3



Basic statistics:


,daily_rank,daily_movement,weekly_movement,popularity,duration_ms,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
count,1.609451e+06,1.609451e+06,1.609451e+06,1.609451e+06,1.609451e+06,1.609451e+06,1.609451e+06,1.609451e+06,1.609451e+06,1.609451e+06,1.609451e+06,1.609451e+06,1.609451e+06,1.609451e+06,1.609451e+06,1.609451e+06,1.609451e+06
mean,2.549174e+01,9.446476e-01,2.864874e+00,7.622463e+01,1.926886e+05,6.817503e-01,6.514109e-01,5.535638e+00,-6.497240e+00,5.391460e-01,9.461374e-02,2.733283e-01,1.793804e-02,1.703592e-01,5.524058e-01,1.222828e+02,3.903617e+00
std,1.442884e+01,7.051024e+00,1.221570e+01,1.566189e+01,4.950747e+04,1.389763e-01,1.638687e-01,3.578172e+00,2.833258e+00,4.984654e-01,9.027822e-02,2.500922e-01,9.477503e-02,1.245026e-01,2.281135e-01,2.817218e+01,3.999104e-01
min,1.000000e+00,-4.900000e+01,-4.900000e+01,0.000000e+00,0.000000e+00,0.000000e+00,2.010000e-05,0.000000e+00,-5.434100e+01,0.000000e+00,0.000000e+00,3.450000e-06,0.000000e+00,1.390000e-02,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.300000e+01,-1.000000e+00,-3.000000e+00,6.500000e+01,1.608120e+05,5.890000e-01,5.520000e-01,2.000000e+00,-7.787000e+00,0.000000e+00,3.880000e-02,6.670000e-02,0.000000e+00,9.590000e-02,3.720000e-01,1.000120e+02,4.000000e+00
50%,2.500000e+01,0.000000e+00,0.000000e+00,8.000000e+01,1.852380e+05,7.020000e-01,6.700000e-01,6.000000e+00,-5.992000e+00,1.000000e+00,5.780000e-02,1.870000e-01,1.220000e-06,1.210000e-01,5.540000e-01,1.199600e+02,4.000000e+00
75%,3.800000e+01,2.000000e+00,5.000000e+00,8.800000e+01,2.174900e+05,7.850000e-01,7.650000e-01,9.000000e+00,-4.693000e+00,1.000000e+00,1.100000e-01,4.360000e-01,7.650000e-05,2.050000e-01,7.370000e-01,1.401080e+02,4.000000e+00
max,5.000000e+01,4.900000e+01,4.900000e+01,1.000000e+02,9.396660e+05,9.880000e-01,9.980000e-01,1.100000e+01,3.233000e+00,1.000000e+00,9.370000e-01,9.960000e-01,9.950000e-01,9.780000e-01,9.920000e-01,2.360890e+02,5.000000e+00


In [6]:
df1 = clean_column_names(df)

/var/folders/k8/0wvkmnks13b_gbyk2zfz4kt40000gn/T/ipykernel_75550/2525275159.py:38: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.translate(accents) if isinstance(x, str) else x)


In [7]:
count_nan_per_column(df1)

,NaN Count,%
Spotify_id,0,0.000000
Name,30,0.001864
Artists,29,0.001802
Daily_rank,0,0.000000
Daily_movement,0,0.000000
Weekly_movement,0,0.000000
Country,21957,1.364254
Snapshot_date,0,0.000000
Popularity,0,0.000000
Is_explicit,0,0.000000


In [8]:
df1

,Spotify_id,Name,Artists,Daily_rank,Daily_movement,Weekly_movement,Country,Snapshot_date,Popularity,Is_explicit,...,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,Time_signature
0,3sK8wGT43QFpWrvNQsrQya,DtMF,Bad Bunny,1,0,8,NaN,2025-01-15,91,True,...,7,-27.405,0,0.0717,0.1770,0.218000,0.0807,0.032,136.020,4
1,2plbrEY59IikOBgBGLjaoe,Die With A Smile,"Lady Gaga, Bruno Mars",2,0,-1,NaN,2025-01-15,100,False,...,6,-7.777,0,0.0304,0.3080,0.000000,0.1220,0.535,157.969,3
2,4wJ5Qq0jBN4ajy7ouZIV1c,APT.,"ROSÉ, Bruno Mars",3,0,-1,NaN,2025-01-15,90,False,...,0,-4.477,0,0.2600,0.0283,0.000000,0.3550,0.939,149.027,4
3,2lTm559tuIvatlT1u0JYG2,BAILE INoLVIDABLE,Bad Bunny,4,0,2,NaN,2025-01-15,90,True,...,10,-46.113,1,0.0615,0.1920,0.790000,0.1120,0.219,119.387,3
4,5TFD2bmFKGhoCRbX61nXY5,NUEVAYoL,Bad Bunny,5,0,0,NaN,2025-01-15,90,False,...,6,-20.024,1,0.1390,0.2650,0.995000,0.2040,0.120,137.922,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1609446,0AYt6NMyyLd0rLuvr0UkMH,Slime You Out (feat. SZA),"Drake, SZA",46,4,0,AE,2023-10-18,84,True,...,5,-9.243,0,0.0502,0.5080,0.000000,0.2590,0.105,88.880,3
1609447,2Gk6fi0dqt91NKvlzGsmm7,SAY MY GRACE (feat. Travis Scott),"Offset, Travis Scott",47,3,0,AE,2023-10-18,80,True,...,10,-5.060,1,0.0452,0.0585,0.000000,0.1320,0.476,121.879,4
1609448,26b3oVLrRUaaybJulow9kz,People,Libianca,48,2,0,AE,2023-10-18,88,False,...,10,-7.621,0,0.0678,0.5510,0.000013,0.1020,0.693,124.357,5
1609449,5ydjxBSUIDn26MFzU3asP4,Rainy Days,V,49,1,0,AE,2023-10-18,88,False,...,9,-8.016,0,0.0875,0.7390,0.000000,0.1480,0.282,74.828,4


In [9]:
def replace_nan_with_glb(df):
    """
    Replaces all NaN values in the 'Country' column with 'GLB'.

    Parameters:
    - df (DataFrame): The input DataFrame containing a 'Country' column.

    Returns:
    - DataFrame: The updated DataFrame with NaN replaced by 'GLB' in the 'Country' column.
    """
    # Ensure the 'Country' column exists
    if 'Country' not in df.columns:
        raise ValueError("The DataFrame must contain a 'Country' column.")
    
    # Replace all NaN with 'GLB' in the 'Country' column
    df['Country'] = df['Country'].fillna('GLB')
    
    return df

In [10]:
df1 = replace_nan_with_glb(df)

In [11]:
def remove_and_store_duplicates(df):
    """
    Identifies and removes exact row duplicates in all columns of a DataFrame.
    
    Prints the count of duplicates and a message after removal. 
    Creates a DataFrame called `df_duplicates` containing the duplicates for later reference.
    
    Parameters:
    - df (DataFrame): The input DataFrame.

    Returns:
    - DataFrame: The updated DataFrame with duplicates removed.
    """
    global df_duplicates

    # Identify duplicate rows
    df_duplicates = df[df.duplicated(keep=False)]
    
    # Count the number of duplicate rows
    duplicate_count = df_duplicates.shape[0]
    
    # Remove duplicates from the DataFrame
    df = df.drop_duplicates(keep='first')
    
    # Print results
    if duplicate_count > 0:
        print(f"{duplicate_count} duplicates found and removed.")
    else:
        print("No duplicates found.")
    print("All duplicates successfully deleted.")
    print("Duplicates can be accessed through df_duplicates")
    return df

In [12]:
df1 = remove_and_store_duplicates(df1)

No duplicates found.
All duplicates successfully deleted.
Duplicates can be accessed through df_duplicates


In [13]:
def enrich_country_data(df):
    """
    Updates the DataFrame by renaming the 'Country' column to 'Country_code', 
    and adding two new columns: 'Country_name' and 'Region', based on the Country_code.

    Parameters:
    - df (DataFrame): The input DataFrame with a 'Country' column.

    Returns:
    - DataFrame: The updated DataFrame with new columns and renamed 'Country' column.
    """
    # Mapping of country codes to names and regions
    country_data = {
        'ZA': ('South Africa', 'Africa'),
        'VN': ('Vietnam', 'Asia'),
        'VE': ('Venezuela', 'America'),
        'UY': ('Uruguay', 'America'),
        'US': ('United States of America', 'America'),
        'UA': ('Ukraine', 'Europe'),
        'TW': ('Taiwan', 'Asia'),
        'TR': ('Turkey', 'Asia'),
        'TH': ('Thailand', 'Asia'),
        'SV': ('El Salvador', 'America'),
        'SK': ('Slovakia', 'Europe'),
        'SG': ('Singapore', 'Asia'),
        'SE': ('Sweden', 'Europe'),
        'SA': ('Saudi Arabia', 'Asia'),
        'RO': ('Romania', 'Europe'),
        'PY': ('Paraguay', 'America'),
        'PT': ('Portugal', 'Europe'),
        'PL': ('Poland', 'Europe'),
        'PK': ('Pakistan', 'Asia'),
        'PH': ('Philippines', 'Asia'),
        'PE': ('Peru', 'America'),
        'PA': ('Panama', 'America'),
        'NZ': ('New Zealand', 'Oceania'),
        'NO': ('Norway', 'Europe'),
        'NL': ('Netherlands', 'Europe'),
        'NI': ('Nicaragua', 'America'),
        'NG': ('Nigeria', 'Africa'),
        'MY': ('Malaysia', 'Asia'),
        'MX': ('Mexico', 'America'),
        'MA': ('Morocco', 'Africa'),
        'LV': ('Latvia', 'Europe'),
        'LU': ('Luxembourg', 'Europe'),
        'LT': ('Lithuania', 'Europe'),
        'KZ': ('Kazakhstan', 'Asia'),
        'KR': ('South Korea', 'Asia'),
        'JP': ('Japan', 'Asia'),
        'IT': ('Italy', 'Europe'),
        'IS': ('Iceland', 'Europe'),
        'IN': ('India', 'Asia'),
        'IL': ('Israel', 'Asia'),
        'IE': ('Ireland', 'Europe'),
        'ID': ('Indonesia', 'Asia'),
        'HU': ('Hungary', 'Europe'),
        'HN': ('Honduras', 'America'),
        'HK': ('Hong Kong', 'Asia'),
        'GT': ('Guatemala', 'America'),
        'GR': ('Greece', 'Europe'),
        'FR': ('France', 'Europe'),
        'FI': ('Finland', 'Europe'),
        'ES': ('Spain', 'Europe'),
        'EG': ('Egypt', 'Africa'),
        'EE': ('Estonia', 'Europe'),
        'EC': ('Ecuador', 'America'),
        'DO': ('Dominican Republic', 'America'),
        'DK': ('Denmark', 'Europe'),
        'DE': ('Germany', 'Europe'),
        'CZ': ('Czech Republic', 'Europe'),
        'CR': ('Costa Rica', 'America'),
        'CO': ('Colombia', 'America'),
        'CL': ('Chile', 'America'),
        'CH': ('Switzerland', 'Europe'),
        'CA': ('Canada', 'America'),
        'BY': ('Belarus', 'Europe'),
        'BR': ('Brazil', 'America'),
        'BO': ('Bolivia', 'America'),
        'BG': ('Bulgaria', 'Europe'),
        'BE': ('Belgium', 'Europe'),
        'AU': ('Australia', 'Oceania'),
        'AT': ('Austria', 'Europe'),
        'AR': ('Argentina', 'America'),
        'AE': ('United Arab Emirates', 'Asia'),
        'GB': ('United Kingdom', 'Europe'),
        'GLB': ('Global', 'Global')
    }

    # Rename the column 'Country' to 'Country_code'
    df = df.rename(columns={'Country': 'Country_code'})

    # Add a new column 'Country_name' using the mapping
    df['Country_name'] = df['Country_code'].map(lambda code: country_data.get(code, ('Unknown', 'Unknown'))[0])

    # Add a new column 'Region' using the mapping
    df['Region'] = df['Country_code'].map(lambda code: country_data.get(code, ('Unknown', 'Unknown'))[1])

    return df

In [14]:
df2 = enrich_country_data(df1)

In [15]:
def download_df_to_csv(df, filename="spot_dash.csv"):
    """
    Saves the given DataFrame to a CSV file in the specified path.
    
    Parameters:
    - df (DataFrame): The DataFrame to save.
    - filename (str): The name of the CSV file (default is 'data.csv').
    """
    # Define the target path
    path = "/Users/ikermontane/Documents/Spotify"
    
    # Ensure the directory exists
    if not os.path.exists(path):
        raise FileNotFoundError(f"The directory '{path}' does not exist.")
    
    # Define the full file path
    file_path = os.path.join(path, filename)
    
    # Save the DataFrame to the CSV file
    df.to_csv(file_path, index=False)
    print(f"DataFrame saved successfully to: {file_path}")

In [16]:
download_df_to_csv(df2)

DataFrame saved successfully to: /Users/ikermontane/Documents/Spotify/spot_dash.csv


In [17]:
import pandas as pd
import pytz
from datetime import datetime, timedelta
from itertools import permutations

def get_effective_date():
    """
    Determines the effective date based on Mexico City time.
    If the current time is before 5:30 PM, it uses yesterday's date.
    """
    # Define the Mexico City timezone
    mexico_city_tz = pytz.timezone("America/Mexico_City")

    # Get the current time in the Mexico City timezone
    now = datetime.now(mexico_city_tz)

    # Determine the effective date
    if now.hour < 17 or (now.hour == 17 and now.minute < 30):
        # Before 5:30 PM, use the previous day's date
        effective_date = now - timedelta(days=1)
    else:
        # After 5:30 PM, use today's date
        effective_date = now

    # Return the date in 'YYYY-MM-DD' format
    return effective_date.strftime('%Y-%m-%d')


def precompute_weekly_averages_by_region_permutations(df2):
    """
    Precomputes weekly averages for the past 365 days grouped by all region permutations.
    Saves the precomputed data into a CSV file for each permutation.
    """
    # Get the effective date based on Mexico City time
    effective_date = datetime.strptime(get_effective_date(), '%Y-%m-%d')

    # Get the past 365 dates
    past_365_dates = [(effective_date - timedelta(days=i)).strftime('%Y-%m-%d') for i in range(365)]

    # Filter data for the past 365 days
    filtered_df = df2[df2['Snapshot_date'].isin(past_365_dates)].copy()  # Use .copy() to avoid SettingWithCopyWarning

    # Ensure the Snapshot_date column is in datetime format
    filtered_df['Snapshot_date'] = pd.to_datetime(filtered_df['Snapshot_date'])

    # Add a "week" column for grouping
    filtered_df['week'] = filtered_df['Snapshot_date'].dt.to_period('W').apply(lambda r: r.start_time)

    # Filter rows where Daily_rank <= 10
    filtered_df = filtered_df[filtered_df['Daily_rank'] <= 10]

    # If the filtered DataFrame is empty, return an empty DataFrame
    if filtered_df.empty:
        print("No data found for the specified date range and criteria.")
        return pd.DataFrame()

    # Get unique regions (excluding Global)
    regions = filtered_df['Region'].dropna().unique().tolist()
    if 'Global' in regions:
        regions.remove('Global')

    # Calculate for Global separately
    global_data = (
        filtered_df[filtered_df['Region'] == 'Global']
        .groupby(['Region', 'week'])
        .agg({
            'Valence': 'mean',
            'Danceability': 'mean',
            'Energy': 'mean'
        })
        .reset_index()
    )

    # Initialize a list to store all permutations' results
    all_permutations_data = [global_data]

    # Iterate over all permutations of regions (1 to N regions)
    for r in range(1, len(regions) + 1):
        for permutation in permutations(regions, r):
            # Filter data for the permutation of regions
            perm_df = filtered_df[filtered_df['Region'].isin(permutation)]

            # Calculate weekly averages for the permutation
            perm_data = (
                perm_df
                .groupby(['week'])
                .agg({
                    'Valence': 'mean',
                    'Danceability': 'mean',
                    'Energy': 'mean'
                })
                .reset_index()
            )

            # Add a "Region" column representing the permutation
            perm_data['Region'] = ', '.join(permutation)

            # Append the result to the list
            all_permutations_data.append(perm_data)

    # Combine all results into a single DataFrame
    final_data = pd.concat(all_permutations_data, ignore_index=True)

    # Save the precomputed data into a CSV file
    file_path = "/Users/ikermontane/Documents/Spotify/precomputed_weekly_data.csv"
    final_data.to_csv(file_path, index=False)
    print(f"Precomputed weekly data saved to: {file_path}")

    return final_data


# Call the function and display the result
weekly_data = precompute_weekly_averages_by_region_permutations(df2)
display(weekly_data)

Precomputed weekly data saved to: /Users/ikermontane/Documents/Spotify/precomputed_weekly_data.csv


,Region,week,Valence,Danceability,Energy
0,Global,2024-01-15,0.534660,0.683980,0.652640
1,Global,2024-01-22,0.543814,0.690457,0.661586
2,Global,2024-01-29,0.516443,0.674186,0.647229
3,Global,2024-02-05,0.484114,0.657871,0.632414
4,Global,2024-02-12,0.461700,0.637314,0.659829
...,...,...,...,...,...
17273,"Oceania, Europe, America, Asia, Africa",2024-12-16,0.574074,0.645106,0.625787
17274,"Oceania, Europe, America, Asia, Africa",2024-12-23,0.587362,0.641062,0.623877
17275,"Oceania, Europe, America, Asia, Africa",2024-12-30,0.564583,0.674925,0.652796
17276,"Oceania, Europe, America, Asia, Africa",2025-01-06,0.483506,0.608132,0.594295
